In [ ]:
# import ros stuff
import rospy
import actionlib
import actionlib.msg
import sys, select
import assignment_2_2022.msg
from nav_msgs.msg import Odometry
from assignment_2_2022.msg import custom_msg
from std_srvs.srv import *
from geometry_msgs.msg import Twist, Pose, Point

import ipywidgets as widgets
import jupyros as jr
import time
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.animation import FuncAnimation
%matplotlib widget

class PositionVisualizer:
    def __init__(self):
        # Init function
        self.fig, self.ax = plt.subplots()
        # Settings for robot's position plot
        self.ln, = plt.plot([], [], 'bo', label='Robot position')
        # Settings for target's position plot
        self.goal_ln, = plt.plot([], [], 'r*', markersize=10, label='Goal position')
        # Robot's position data arrays
        self.x_data, self.y_data = [], []
    
    def plot_init(self):
        # Set axis limits
        self.ax.set_xlim(10, -10)
        self.ax.set_ylim(10, -10)
        # Set the grid
        self.ax.grid(True, color='lightgrey')
        # Set the title
        self.ax.set_title('Robot position')
        # Set the legend
        self.ax.legend(loc='upper left')
        
        return self.ln, self.goal_ln
    
    def odom_callback(self, message):
        # Callback function to update the data arrays
        self.x_data.append(message.pose.pose.position.x)
        self.y_data.append(message.pose.pose.position.y)
    
    def update_plot(self, frame):
        # Update the robot position plot
        self.ln.set_data(self.x_data, self.y_data)

        if goal is not None:
            self.goal_ln.set_data(goal.target_pose.pose.position.x, goal.target_pose.pose.position.y)
        else:
            self.goal_ln.set_data([], [])
        return self.ln, self.goal_ln

def action_client():
    global goal

    # Creation of the goal for the robot
    goal = assignment_2_2022.msg.PlanningGoal()
    goal.target_pose.pose.position.x = w_coordinate_x.value
    goal.target_pose.pose.position.y = w_coordinate_y.value

    # Send goal to the server
    act_client.send_goal(goal)

def publish_msg(message):
   
    global publisher
    global last_time_published_odom
        
    # Get the position and velocity
    pos_x = message.pose.pose.position.x
    pos_y = message.pose.pose.position.y
    vel_x = message.twist.twist.linear.x
    vel_y = message.twist.twist.linear.y

    # Creation of the custom message
    custom_message = custom_msg()
    
    custom_message.actual_x = pos_x
    custom_message.actual_y = pos_y
    custom_message.actual_vel_x = vel_x
    custom_message.actual_vel_y = vel_y
    
    # Publishing the custom message
    publisher.publish(custom_message)
    
    current_time = time.time() * 1000
    if current_time - last_time_published_odom > 100:
        print("\rRobot position: x={}, y={}".format(pos_x, pos_y), end='')
        last_time_published_odom = current_time
        
def on_send_goal_button_clicked(b):
    # Call 'action_cliient' function
    action_client()

def on_cancel_goal_button_clicked(b):
    global goal
    
    # Cancel the goal
    act_client.cancel_goal()
    
    goal = None
    
if __name__ == '__main__':

    # Creation of the node A
    rospy.init_node('Node_A')

    # Creation of the Publisher
    publisher = rospy.Publisher('/position_and_velocity', custom_msg, queue_size = 1)
    
    # Creation of the action client
    act_client = actionlib.SimpleActionClient('/reaching_goal', assignment_2_2022.msg.PlanningAction)

    # Waiting the server
    act_client.wait_for_server()
    
    # Widgets to give the goal coordinates
    w_coordinate_x = widgets.FloatText(description='Coordinate X:', step = 0.1)
    w_coordinate_y = widgets.FloatText(description='Coordinate Y:', step = 0.1)

    # Button to send the goal
    send_goal_button = widgets.Button(
    description='Send goal to the robot',
    layout = widgets.Layout(width = 'auto'))
    
    send_goal_button.on_click(on_send_goal_button_clicked)

    # Button to cancel the goal
    cancel_goal_button = widgets.Button(
    description='Cancel goal sent to the robot',
    layout = widgets.Layout(width = 'auto'))
    
    cancel_goal_button.on_click(on_cancel_goal_button_clicked)

    display(widgets.HBox([w_coordinate_x, w_coordinate_y]))
    display(widgets.HBox([send_goal_button, cancel_goal_button]))
    
    # Initialization
    goal = None
    
    last_time_published_odom = 0
    jr.subscribe('/odom', Odometry, publish_msg)
    
    # Create the visualizer object
    position_visualizer = PositionVisualizer()
    
    # Subscriber for the odom topic
    sub = rospy.Subscriber('/odom', Odometry, position_visualizer.odom_callback)

    # Plot
    position_animation = FuncAnimation(
        position_visualizer.fig,
        position_visualizer.update_plot,
        init_func = position_visualizer.plot_init,
        #save_count = None)  
        cache_frame_data=False)
    plt.show(block = True)